In [93]:
import pandas as pd
import numpy as np
import seaborn as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler



In [94]:
df=pd.read_csv('train.csv',usecols=['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [95]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [96]:


X=df.drop(['Survived'],axis=1)
Y=df['Survived']

In [97]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [98]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S


In [99]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         141
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [100]:
# 1) impute
# 2) encode
# 3) scale
# 4) feature select
# 5) model

In [101]:
Si=SimpleImputer(strategy='most_frequent')
Si2=SimpleImputer(strategy='mean')
ohe=OneHotEncoder(handle_unknown='ignore')
# handle_unknown='ignore'---Whether to raise an error or ignore if an unknown categorical feature is present during transform (default is to raise).
min=MaxAbsScaler()



In [102]:
trf1=make_column_transformer(
     (Si,[6]),
    (Si2,[2]),remainder='passthrough'
)

In [103]:
trf2=make_column_transformer(
    (ohe,[1,6])
)

In [104]:
trf3=make_column_transformer(
    (min, [0,1,2,3,4,5,6])
)

In [105]:
#During passing model throgh pipeline direct used pip.fit(X_train)at th end
#if no model is paassed ,only preproceesing technique is used then used pipe.fit_transform(X_train)

In [106]:
from sklearn.tree import DecisionTreeClassifier
trf4 = DecisionTreeClassifier()

In [107]:
pipe=make_pipeline(trf1,trf2,trf3,trf4)

In [108]:
from sklearn import set_config
set_config(display='diagram')

In [109]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('simpleimputer-1',
                                  SimpleImputer(strategy='most_frequent'), [6]),
                                 ('simpleimputer-2', SimpleImputer(), [2])]),
 'columntransformer-2': ColumnTransformer(transformers=[('onehotencoder',
                                  OneHotEncoder(handle_unknown='ignore'),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('maxabsscaler', MaxAbsScaler(),
                                  [0, 1, 2, 3, 4, 5, 6])]),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [110]:

pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simpleimputer-1',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6]),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(), [2])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('maxabsscaler',
                                                  MaxAbsScaler(),
                                                  [0, 1, 2, 3, 4, 5, 6])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [111]:
y_pred = pipe.predict(X_test)

In [112]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

In [117]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6320397911947208